In [450]:
%reset -f

import numpy as np
import pandas as pd
import time
import os

In [451]:
ACTIONS = ['up', 'down', 'left', 'right']     # available actions
EPSILON = 0.9   # greedy police
ALPHA = 0.1     # learning rate
GAMMA = 0.9    # discount factor
MAX_EPISODES = 35   # maximum episodes
FRESH_TIME = 0.1    # fresh time for one move

# Set map
row = 8
col = 9
testmap = np.zeros((row,col))
# set outside wall
testmap[:, 0] = 1
testmap[:, col-1] = 1
testmap[0, :] = 1
testmap[row-1, :] = 1

# set entry and exit
my_entry = [row-1, 1]
my_exit = [1, col-1]
testmap[row-1, 1] = 0
testmap[1, col-1] = 0

In [452]:
def clear():
    os.system('cls')

In [453]:
# Define Map class
class MazeMap(object):
    def __init__(self, row, col, mymap, entry, exit):
        self.maze_wall = '#'
        self.maze_road = ' '
        self.maze_person_mark = 'T'
        
        
        self.row = row
        self.col = col
        self.mymap = mymap    #ndarray
        
        self.entry = entry
        self.exit = exit
        
        self.my_person=Person(entry)
     
       
    def drawMap(self, person_coor=None):
        if person_coor is None:
            person_coor = self.my_person.coor
        #print coor
        print(0, end='')
        for i in range(self.col-1):
            print(i, end='')
        print(self.col-1)
        
        #print maze
        for i in range(self.row):
            for j in range(self.col):
                if (person_coor[0]==i and person_coor[1]==j):    #coor is person
                    if j == self.col-1:
                        print(self.maze_person_mark)
                        continue
                    elif j == 0:
                        print(i, end='')
                        print(self.maze_person_mark, end='')
                        continue
                    else:
                        print(self.maze_person_mark, end='')
                        continue
                        
                if j == self.col - 1:
                    if self.isDoor([i,j])==-1:
                        print(self.maze_wall)
                    else:
                        print(self.maze_road)
                elif j == 0:
                    if self.isDoor([i,j])==-1:
                        print(i,end = '')
                        print(self.maze_wall, end = '')
                    else:
                        print(i,end = '')
                        print(self.maze_road, end = '')
                if j!=0 and j!=self.col-1:
                    if self.mymap[i, j] == 1:
                        print(self.maze_wall, end = '')
                    else:
                        print(self.maze_road, end = '')
    def isWall(self, coor):
        x = coor[0]
        y = coor[1]
        self.wall_state = [-1,-1,-1,-1] #up,down,left,right initial
                
        if (x<0 or x>self.row-1) or (y<0 or y>self.col-1):    #false
            return self.wall_state
        
        #conner
        if x==0 and y==0:
            self.wall_state[0] =-1         
            if self.mymap[x+1, y]==0:
                self.wall_state[1] =0
            else:
                self.wall_state[1] =1
                
            self.wall_state[2] =-1
            
            if self.mymap[x, y+1]==0:
                self.wall_state[3] =0
            else:
                self.wall_state[3] =1
            return self.wall_state                
        if x==self.row-1 and y==0:
            if self.mymap[x-1, y]==0:
                self.wall_state[0] =0
            else:
                self.wall_state[0] =1
                
            self.wall_state[1] =-1
            
            self.wall_state[2] =-1
            
            if self.mymap[x, y+1]==0:
                self.wall_state[3] =0
            else:
                self.wall_state[3] =1
            return self.wall_state        
        if x==self.row-1 and y==self.col-1:
            if self.mymap[x-1, y]==0:
                self.wall_state[0] =0
            else:
                self.wall_state[0] =1        
            self.wall_state[1] =-1
            if self.mymap[x, y-1]==0:
                self.wall_state[2] =0
            else:
                self.wall_state[2] =1
            self.wall_state[3] =-1
            return self.wall_state
        if x==0 and y==self.col-1:
            self.wall_state[0] =-1
            if self.mymap[x+1, y]==0:
                self.wall_state[1] =0
            else:
                self.wall_state[1] =1
            if self.mymap[x, y-1]==0:
                self.wall_state[2] =0
            else:
                self.wall_state[2] =1
            self.wall_state[3] =-1
            return self.wall_state

        #Wall
        if x==0: #top -1,?,1,1
            self.wall_state[0] =-1
            
            if self.mymap[x+1, y]==0:
                self.wall_state[1] =0
            else:
                self.wall_state[1] =1
                
            if self.mymap[x, y-1]==0:
                self.wall_state[2] =0
            else:
                self.wall_state[2] =1
        
            if self.mymap[x, y+1]==0:
                self.wall_state[3] =0
            else:
                self.wall_state[3] =1
            return self.wall_state
        if x==self.row-1: #bottom ?,-1,1,1
            if self.mymap[x-1, y]==0:
                self.wall_state[0] =0
            else:
                self.wall_state[0] =1
                
            self.wall_state[1] =-1
            
            if self.mymap[x, y-1]==0:
                self.wall_state[2] =0
            else:
                self.wall_state[2] =1
        
            if self.mymap[x, y+1]==0:
                self.wall_state[3] =0
            else:
                self.wall_state[3] =1
            return self.wall_state
        if y==0: #left 1,1,-1,?
            if self.mymap[x-1, y]==0:
                self.wall_state[0] =0
            else:
                self.wall_state[0] =1
                
            if self.mymap[x+1, y]==0:
                self.wall_state[1] =0
            else:
                self.wall_state[1] =1
            
            self.wall_state[2] =-1
            
            if self.mymap[x, y+1]==0:
                self.wall_state[3] =0
            else:
                self.wall_state[3] =1
            return self.wall_state
        if y==self.col-1: #right 1,1,?,-1
            if self.mymap[x-1, y]==0:
                self.wall_state[0] =0
            else:
                self.wall_state[0] =1
                
            if self.mymap[x+1, y]==0:
                self.wall_state[1] =0
            else:
                self.wall_state[1] =1
            
            if self.mymap[x, y-1]==0:
                self.wall_state[2] =0
            else:
                self.wall_state[2] =1
                
            self.wall_state[3] =-1
            return self.wall_state
        
        # inside the maze
        if self.mymap[x-1, y]==0:
            self.wall_state[0] =0
        else:
            self.wall_state[0] =1
            
        if self.mymap[x+1, y]==0:
            self.wall_state[1] =0
        else:
            self.wall_state[1] =1
        
        if self.mymap[x, y-1]==0:
            self.wall_state[2] =0
        else:
            self.wall_state[2] =1
        
        if self.mymap[x, y+1]==0:
            self.wall_state[3] =0
        else:
            self.wall_state[3] =1
        return self.wall_state
        
    def isDoor(self, coor):
        if coor == self.entry:
            return 0
        elif coor == self.exit:
            return 1
        else:
            return -1
    
    def movePerson(self, way):
        if way=='up':
            if self.isWall(self.my_person.coor)[0] == 0:
                self.my_person.coor[0] -=1
        if way=='down':
            if self.isWall(self.my_person.coor)[1] == 0:
                self.my_person.coor[0] +=1
        if way=='left':
            if self.isWall(self.my_person.coor)[2] == 0:
                self.my_person.coor[1] -=1
        if way=='right':
            if self.isWall(self.my_person.coor)[3] == 0:
                self.my_person.coor[1] +=1
                
        #update person's state
        S_ = self.my_person.coor
        #give reward
        if S_ == self.exit:
            R = 1
        else:
            R = 0
        return S_, R

In [454]:
# Define Person Class
class Person(object):
    def __init__(self,entry):
        self.coor = entry

In [455]:
# Define q Table
def build_q_table(coor, actions):
    tmp = coor
    tmp.append( len(actions))
    table = np.zeros(tmp) # q_table initial values
    #print(table)    # show table
    return table

In [456]:
# Define how to choose an action
def choose_action(coor, q_table):
    state_actions = q_table[coor[0], coor[1], :]       #[0, 0, 0, 0]
    if (np.random.uniform() > EPSILON) or ((state_actions == 0).all()): 
        action_name = np.random.choice(ACTIONS)
    else:   # act greedy
        index = state_actions.tolist().index(max(state_actions))
        action_name = ACTIONS[index]
    return action_name

In [457]:
def getIndex(A):
    if A == 'up':
        return 0
    if A == 'down':
        return 1
    if A == 'left':
        return 2
    if A == 'right':
        return 3

In [458]:
#Reinforcement Learning
def RL():
    q_table = build_q_table([row, col], ACTIONS)
    for episode in range(MAX_EPISODES):
        print(type(q_table))
        print(q_table)
        step_counter = 0
        S = my_entry
        is_terminated = False
        myMaze = MazeMap(row, col, testmap, my_entry, my_exit)
        myMaze.drawMap()
        print('round %d' % episode)
        while not is_terminated:
            print('current coor is: %d, %d' % (S[0],S[1]))
            A = choose_action(S, q_table)
            print('my entry is: %d, %d' % (my_entry[0],my_entry[1]))
            S_, R = myMaze.movePerson(A)    # take action & get next state and reward
            print('my entry is: %d, %d' % (my_entry[0],my_entry[1]))
            tmp = getIndex(A)
            q_predict = q_table[S[0], S[1], tmp]
            if S_ != my_exit:
                q_target = R + GAMMA * q_table[S_[0],S[1], :].max()   # next state is not terminal
            else:
                q_target = R     # next state is terminal
                is_terminated = True    # terminate this episode

            q_table[S[0], S[1], tmp] += ALPHA * (q_target - q_predict)  # update
            S = S_  # move to next state!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            print('next coor is: %d, %d' % (S[0],S[1]))
            print(is_terminated)
            clear()
            myMaze.drawMap()
            step_counter += 1
    return q_table

In [459]:
if __name__ == "__main__":
    q_table = RL()

    print('\r\nQ-table:\n')
    print(q_table)

<class 'numpy.ndarray'>
[[[ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]

 [[ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]

 [[ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]

 [[ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]

 [[ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]

 [[ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.

2#T      #
3#       #
4#       #
5#       #
6#       #
7# #######
current coor is: 2, 1
my entry is: 2, 1
my entry is: 2, 2
next coor is: 2, 2
False
0012345678
0#########
1#        
2# T     #
3#       #
4#       #
5#       #
6#       #
7# #######
current coor is: 2, 2
my entry is: 2, 2
my entry is: 2, 1
next coor is: 2, 1
False
0012345678
0#########
1#        
2#T      #
3#       #
4#       #
5#       #
6#       #
7# #######
current coor is: 2, 1
my entry is: 2, 1
my entry is: 2, 1
next coor is: 2, 1
False
0012345678
0#########
1#        
2#T      #
3#       #
4#       #
5#       #
6#       #
7# #######
current coor is: 2, 1
my entry is: 2, 1
my entry is: 1, 1
next coor is: 1, 1
False
0012345678
0#########
1#T       
2#       #
3#       #
4#       #
5#       #
6#       #
7# #######
current coor is: 1, 1
my entry is: 1, 1
my entry is: 1, 1
next coor is: 1, 1
False
0012345678
0#########
1#T       
2#       #
3#       #
4#       #
5#       #
6#       #
7# #######
current coor is: 1, 1
my

0012345678
0#########
1#       T
2#       #
3#       #
4#       #
5#       #
6#       #
7# #######
<class 'numpy.ndarray'>
[[[ 0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.  ]]

 [[ 0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.19]]

 [[ 0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.  ]]

 [[ 0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.  ]
  [

0012345678
0#########
1#       T
2#       #
3#       #
4#       #
5#       #
6#       #
7# #######
round 5
current coor is: 1, 8
my entry is: 1, 8
my entry is: 1, 8
next coor is: 1, 8
True
0012345678
0#########
1#       T
2#       #
3#       #
4#       #
5#       #
6#       #
7# #######
<class 'numpy.ndarray'>
[[[ 0.        0.        0.        0.      ]
  [ 0.        0.        0.        0.      ]
  [ 0.        0.        0.        0.      ]
  [ 0.        0.        0.        0.      ]
  [ 0.        0.        0.        0.      ]
  [ 0.        0.        0.        0.      ]
  [ 0.        0.        0.        0.      ]
  [ 0.        0.        0.        0.      ]
  [ 0.        0.        0.        0.      ]]

 [[ 0.        0.        0.        0.      ]
  [ 0.        0.        0.        0.      ]
  [ 0.        0.        0.        0.      ]
  [ 0.        0.        0.        0.      ]
  [ 0.        0.        0.        0.      ]
  [ 0.        0.        0.        0.      ]
  [ 0.        0.        0.

6#       #
7# #######
round 10
current coor is: 1, 8
my entry is: 1, 8
my entry is: 1, 8
next coor is: 1, 8
True
0012345678
0#########
1#       T
2#       #
3#       #
4#       #
5#       #
6#       #
7# #######
<class 'numpy.ndarray'>
[[[ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]]

 [[ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.    

...

4#       #
5#       #
6#       #
7# #######
current coor is: 2, 6
my entry is: 2, 6
my entry is: 3, 6
next coor is: 3, 6
False
0012345678
0#########
1#        
2#       #
3#     T #
4#       #
5#       #
6#       #
7# #######
current coor is: 3, 6
my entry is: 3, 6
my entry is: 3, 5
next coor is: 3, 5
False
0012345678
0#########
1#        
2#       #
3#    T  #
4#       #
5#       #
6#       #
7# #######
current coor is: 3, 5
my entry is: 3, 5
my entry is: 3, 4
next coor is: 3, 4
False
0012345678
0#########
1#        
2#       #
3#   T   #
4#       #
5#       #
6#       #
7# #######
current coor is: 3, 4
my entry is: 3, 4
my entry is: 4, 4
next coor is: 4, 4
False
0012345678
0#########
1#        
2#       #
3#       #
4#   T   #
5#       #
6#       #
7# #######
current coor is: 4, 4
my entry is: 4, 4
my entry is: 3, 4
next coor is: 3, 4
False
0012345678
0#########
1#        
2#       #
3#   T   #
4#       #
5#       #
6#       #
7# #######
current coor is: 3, 4
my entry is: 3, 4
my ent

7# #######
current coor is: 5, 6
my entry is: 5, 6
my entry is: 5, 7
next coor is: 5, 7
False
0012345678
0#########
1#        
2#       #
3#       #
4#       #
5#      T#
6#       #
7# #######
current coor is: 5, 7
my entry is: 5, 7
my entry is: 4, 7
next coor is: 4, 7
False
0012345678
0#########
1#        
2#       #
3#       #
4#      T#
5#       #
6#       #
7# #######
current coor is: 4, 7
my entry is: 4, 7
my entry is: 5, 7
next coor is: 5, 7
False
0012345678
0#########
1#        
2#       #
3#       #
4#       #
5#      T#
6#       #
7# #######
current coor is: 5, 7
my entry is: 5, 7
my entry is: 6, 7
next coor is: 6, 7
False
0012345678
0#########
1#        
2#       #
3#       #
4#       #
5#       #
6#      T#
7# #######
current coor is: 6, 7
my entry is: 6, 7
my entry is: 6, 7
next coor is: 6, 7
False
0012345678
0#########
1#        
2#       #
3#       #
4#       #
5#       #
6#      T#
7# #######
current coor is: 6, 7
my entry is: 6, 7
my entry is: 6, 7
next coor is: 6, 7
Fa

0012345678
0#########
1#        
2#       #
3#       #
4#       #
5#       #
6#     T #
7# #######
current coor is: 6, 6
my entry is: 6, 6
my entry is: 6, 7
next coor is: 6, 7
False
0012345678
0#########
1#        
2#       #
3#       #
4#       #
5#       #
6#      T#
7# #######
current coor is: 6, 7
my entry is: 6, 7
my entry is: 6, 7
next coor is: 6, 7
False
0012345678
0#########
1#        
2#       #
3#       #
4#       #
5#       #
6#      T#
7# #######
current coor is: 6, 7
my entry is: 6, 7
my entry is: 5, 7
next coor is: 5, 7
False
0012345678
0#########
1#        
2#       #
3#       #
4#       #
5#      T#
6#       #
7# #######
current coor is: 5, 7
my entry is: 5, 7
my entry is: 5, 6
next coor is: 5, 6
False
0012345678
0#########
1#        
2#       #
3#       #
4#       #
5#     T #
6#       #
7# #######
current coor is: 5, 6
my entry is: 5, 6
my entry is: 5, 5
next coor is: 5, 5
False
0012345678
0#########
1#        
2#       #
3#       #
4#       #
5#    T  #
6#       #
7#

7# #######
current coor is: 3, 7
my entry is: 3, 7
my entry is: 4, 7
next coor is: 4, 7
False
0012345678
0#########
1#        
2#       #
3#       #
4#      T#
5#       #
6#       #
7# #######
current coor is: 4, 7
my entry is: 4, 7
my entry is: 5, 7
next coor is: 5, 7
False
0012345678
0#########
1#        
2#       #
3#       #
4#       #
5#      T#
6#       #
7# #######
current coor is: 5, 7
my entry is: 5, 7
my entry is: 4, 7
next coor is: 4, 7
False
0012345678
0#########
1#        
2#       #
3#       #
4#      T#
5#       #
6#       #
7# #######
current coor is: 4, 7
my entry is: 4, 7
my entry is: 4, 6
next coor is: 4, 6
False
0012345678
0#########
1#        
2#       #
3#       #
4#     T #
5#       #
6#       #
7# #######
current coor is: 4, 6
my entry is: 4, 6
my entry is: 4, 5
next coor is: 4, 5
False
0012345678
0#########
1#        
2#       #
3#       #
4#    T  #
5#       #
6#       #
7# #######
current coor is: 4, 5
my entry is: 4, 5
my entry is: 5, 5
next coor is: 5, 5
Fa

0012345678
0#########
1#        
2#       #
3#       #
4# T     #
5#       #
6#       #
7# #######
current coor is: 4, 2
my entry is: 4, 2
my entry is: 4, 3
next coor is: 4, 3
False
0012345678
0#########
1#        
2#       #
3#       #
4#  T    #
5#       #
6#       #
7# #######
current coor is: 4, 3
my entry is: 4, 3
my entry is: 4, 4
next coor is: 4, 4
False
0012345678
0#########
1#        
2#       #
3#       #
4#   T   #
5#       #
6#       #
7# #######
current coor is: 4, 4
my entry is: 4, 4
my entry is: 4, 5
next coor is: 4, 5
False
0012345678
0#########
1#        
2#       #
3#       #
4#    T  #
5#       #
6#       #
7# #######
current coor is: 4, 5
my entry is: 4, 5
my entry is: 3, 5
next coor is: 3, 5
False
0012345678
0#########
1#        
2#       #
3#    T  #
4#       #
5#       #
6#       #
7# #######
current coor is: 3, 5
my entry is: 3, 5
my entry is: 4, 5
next coor is: 4, 5
False
0012345678
0#########
1#        
2#       #
3#       #
4#    T  #
5#       #
6#       #
7#

In [426]:
my_entry

[1, 8]